In [5]:
import pandas as pd
import glob

## 1. Setup the combined CDFs

In [22]:
def merge_csv_files(file_paths):
    """
    Diese Funktion lädt mehrere CSV-Dateien und kombiniert sie, indem sie die Werte für gleiche Spalten untereinander in neue Zeilen schreibt.

    :param file_paths: Liste der Dateipfade zu den CSV-Dateien
    :return: Zusammengeführter DataFrame
    """
    dataframes = [pd.read_csv(file) for file in file_paths]
    merged_df = pd.concat(dataframes, ignore_index=True)
    return merged_df

file_paths = ["saved_csv/CDF_Functions_Radiation.csv",
              "saved_csv/CDF_Functions_Temperature.csv",]
              #"saved_csv/CDF_Functions_Usage.csv"]
merged_df = merge_csv_files(file_paths)

merged_df.to_csv("saved_csv/CDF_Functions.csv", index=False)


## 2. Read the combined CDFs correctly

``CDF: 589 float values in list``

``Radiation_CDF: df["Month_X_Hour_Y"][0]``

``Temperature_CDF: df["Month_X_Hour_Y"][1]``

``X = 1, ..., 12; Y = 0, ..., 23``

In [35]:
file_path = "saved_csv/CDF_Functions.csv"

# CSV einlesen
df = pd.read_csv(file_path)

# Umwandlung der CDF-Spalten von Strings zu Listen von Floats
for column in df.columns[1:]:  # Erste Spalte ist "CDFs", daher erst ab Spalte 1
    df[column] = df[column].apply(lambda x: list(map(float, x.split(";"))) if isinstance(x, str) else [])

589


## Calculate Correlation

In [ ]:
df = pd.read_csv("data/Solar_Timeseries_2005_2023.csv")
# Zeitstempel umwandeln
df['time'] = pd.to_datetime(df['time'], format='%Y%m%d:%H%M', errors='coerce')

# Relevante Spalten auswählen
df_filtered = df[['time', 'G(i) (Globalstrahlung)', 'T2m (Temperatur)']].dropna()

# Korrelation zwischen Globalstrahlung und Temperatur berechnen
correlation = df_filtered[['G(i) (Globalstrahlung)', 'T2m (Temperatur)']].corr()
correlation


,G(i) (Globalstrahlung),T2m (Temperatur)
G(i) (Globalstrahlung),1.000000,0.378649
T2m (Temperatur),0.378649,1.000000


In [36]:
import numpy as np
import pandas as pd
from scipy.stats import norm

def generate_correlated_samples(n_samples, correlation, radiation_cdf, temperature_cdf):
    """
    Generiert n_samples Samples, die mittels Gaussian Copula so korreliert werden,
    dass die marginalen Verteilungen den vorgegebenen CDFs (als Quantile) entsprechen.

    Parameter
    ----------
    n_samples : int
        Anzahl der zu generierenden Samples.
    correlation : float
        Gewünschter Korrelationswert zwischen Globalstrahlung und Temperatur.
    radiation_cdf : list oder np.array
        Sortierte Liste von Werten der Strahlungsverteilung (589 Werte).
    temperature_cdf : list oder np.array
        Sortierte Liste von Werten der Temperaturverteilung (589 Werte).

    Returns
    -------
    radiation_samples : np.array
        Samples aus der Strahlungsverteilung.
    temperature_samples : np.array
        Samples aus der Temperaturverteilung.
    """
    # Kovarianzmatrix für 2 Variablen
    cov = np.array([[1, correlation],
                    [correlation, 1]])

    # Generiere Samples aus der multivariaten Normalverteilung
    mv_samples = np.random.multivariate_normal(mean=[0, 0], cov=cov, size=n_samples)

    # Transformation in Uniformvariablen über die Standard-Normal-CDF
    u = norm.cdf(mv_samples)

    # Erstelle äquidistante Wahrscheinlichkeiten, die den Quantilen der CDFs entsprechen
    p_radiation = np.linspace(0, 1, len(radiation_cdf))
    p_temperature = np.linspace(0, 1, len(temperature_cdf))

    # Inverse Transform Sampling mittels Interpolation
    radiation_samples = np.interp(u[:, 0], p_radiation, radiation_cdf)
    temperature_samples = np.interp(u[:, 1], p_temperature, temperature_cdf)

    return radiation_samples, temperature_samples

def generate_samples_for_hour_from_df(df, column, n_samples=1000, correlation=0.3786):
    """
    Generiert für die angegebene Stunde (Spalte) korrelierte Samples aus dem
    bereits geladenen DataFrame.

    Annahmen:
      - Der DataFrame enthält Spalten mit Namen wie "Month_X_Hour_Y"
        (X = 1, ..., 12; Y = 0, ..., 23).
      - In jeder dieser Spalten:
            Zeile 0 enthält die Radiation_CDF (Liste von 589 floats)
            Zeile 1 enthält die Temperature_CDF (Liste von 589 floats)

    Parameter
    ----------
    df : pd.DataFrame
        DataFrame, der die CDFs enthält.
    column : str
        Name der Spalte, z.B. "Month_4_Hour_13".
    n_samples : int, optional
        Anzahl der zu generierenden Samples (Standard: 1000).
    correlation : float, optional
        Gewünschter Korrelationswert (Standard: 0.3786).

    Returns
    -------
    radiation_samples, temperature_samples : tuple von np.array
        Korrelierte Samples aus der Strahlungs- und Temperaturverteilung.
    """
    radiation_cdf = df[column][0]
    temperature_cdf = df[column][1]

    return generate_correlated_samples(n_samples, correlation, radiation_cdf, temperature_cdf)


# Beispielaufruf für "Month_4_Hour_13":
samples_rad, samples_temp = generate_samples_for_hour_from_df(df, "Month_4_Hour_13", n_samples=1000, correlation=0.3786)

# Optional: Ausgabe der ersten 10 Samples
print("Radiation Samples:", samples_rad[:10])
print("Temperature Samples:", samples_temp[:10])


Radiation Samples: [121.09710185 211.55480841 107.08612446 474.64651504 561.56619769
 485.29411037 891.05603966 660.36670471 959.16466731 297.00029994]
Temperature Samples: [ 8.8809792  12.31000496 16.13       17.53597052 15.53       18.01458442
 12.06685529 13.90889791 22.87131265 15.2563865 ]
